#XÂY DỰNG MÔI TRƯỜNG

In [ ]:
# # Môi trường lập trình
# from google.colab import drive
# drive.mount("/content/gdrive")
# folder = '/content/gdrive/My Drive/Colab Notebooks/DATASET_DAP/Output'

In [ ]:
!pip install pyfpgrowth

In [ ]:
# Thư viện cần sử dụng
import pandas as pd
import numpy as np
import pyfpgrowth
from prettytable import PrettyTable

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# https://drive.google.com/file/d/1P4ILFVOrmmYH6BibikRytI1cbWa_RKas/view?usp=sharing
!gdown --id 1P4ILFVOrmmYH6BibikRytI1cbWa_RKas

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1P4ILFVOrmmYH6BibikRytI1cbWa_RKas
To: /content/weatherAUS_cleaned_3.1.csv
100% 22.5M/22.5M [00:00<00:00, 88.0MB/s]


In [ ]:
# Đọc dữ liệu từ file CSV
data = pd.read_csv('weatherAUS_cleaned_3.1.csv')
df = data.copy()
df.head(5)

,Date,Location,MinTemp,MaxTemp,Rainfall,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,RainTomorrow,is_RainToday,is_RainTomorrow,State/Territory,Postal,Region Type,Region,Month,Year,Season
0,2008-12-01,Albury,13.4,22.9,0.6,7.624853,W,44.0,W,WNW,...,No,0.0,0.0,New South Wales,NSW,State,Southeast,12,2008,Summer
1,2008-12-02,Albury,7.4,25.1,0.0,7.624853,WNW,44.0,NNW,WSW,...,No,0.0,0.0,New South Wales,NSW,State,Southeast,12,2008,Summer
2,2008-12-03,Albury,12.9,25.7,0.0,7.624853,WSW,46.0,W,WSW,...,No,0.0,0.0,New South Wales,NSW,State,Southeast,12,2008,Summer
3,2008-12-04,Albury,9.2,28.0,0.0,7.624853,NE,24.0,SE,E,...,No,0.0,0.0,New South Wales,NSW,State,Southeast,12,2008,Summer
4,2008-12-05,Albury,17.5,32.3,1.0,7.624853,W,41.0,ENE,NW,...,No,0.0,0.0,New South Wales,NSW,State,Southeast,12,2008,Summer


#XÂY DỰNG CỘT PHÂN TÍCH

##Hàm khởi tạo

In [ ]:
#Tính giá trị IQR cho từng cột
def calculate_quartiles(column_name, df):
    quartiles = df[column_name].quantile([0.25, 0.5, 0.75])
    return quartiles.values

In [ ]:
#Xây dựng giá trị định tính tạm thời cho từng cột
def temporary_category(value, quartiles):
    if value <= quartiles[0]:
        return 'Low'
    elif quartiles[0] < value <= quartiles[1]:
        return 'Low Average'
    elif quartiles[1] < value <= quartiles[2]:
        return 'High Average'
    else:
        return 'High'

In [ ]:
#Xây dựng giá trị cho cột tổng hợp của 2 biến định lượng
def determine_category(row, column_9am, column_3pm):
    if row[column_9am] == 'Low' and row[column_3pm] == 'Low':
        return 'Low'
    elif row[column_9am] == 'High' and row[column_3pm] == 'High':
        return 'High'
    elif row[column_9am] == 'Low Average' and row[column_3pm] == 'Low Average':
        return 'Low Average'
    elif row[column_9am] == 'High Average' and row[column_3pm] == 'High Average':
        return 'High Average'
    elif (row[column_9am] == 'Low' and row[column_3pm] == 'Low Average') or (row[column_9am] == 'Low Average' and row[column_3pm] == 'Low'):
        return 'Low'
    elif (row[column_9am] == 'Low' and row[column_3pm] == 'High Average') or (row[column_9am] == 'High Average' and row[column_3pm] == 'Low'):
        return 'Low Average'
    elif (row[column_9am] == 'Low' and row[column_3pm] == 'High') or (row[column_9am] == 'High' and row[column_3pm] == 'Low'):
        return 'Average'
    elif (row[column_9am] == 'Low Average' and row[column_3pm] == 'High Average') or (row[column_9am] == 'High Average' and row[column_3pm] == 'Low Average'):
        return 'Average'
    elif (row[column_9am] == 'Low Average' and row[column_3pm] == 'High') or (row[column_9am] == 'High' and row[column_3pm] == 'Low Average'):
        return 'High Average'
    elif (row[column_9am] == 'High Average' and row[column_3pm] == 'High') or (row[column_9am] == 'High' and row[column_3pm] == 'High Average'):
        return 'High'

##Thêm mới cột WindSpeed




In [ ]:
quartiles_wind_speed_9am = calculate_quartiles('WindSpeed9am', df)
quartiles_wind_speed_3pm = calculate_quartiles('WindSpeed3pm', df)

print(f'Giá trị IQR của cột WindSpeed9am',quartiles_wind_speed_9am)
print(f'Giá trị IQR của cột WindSpeed3pm',quartiles_wind_speed_3pm)

Giá trị IQR của cột WindSpeed9am [ 7. 13. 19.]
Giá trị IQR của cột WindSpeed3pm [13. 17. 24.]


In [ ]:
df['WindSpeed9am'] = df.apply(lambda row: temporary_category(row['WindSpeed9am'], quartiles_wind_speed_9am), axis=1)
df['WindSpeed3pm'] = df.apply(lambda row: temporary_category(row['WindSpeed3pm'], quartiles_wind_speed_3pm), axis=1)

In [ ]:
df[['WindSpeed9am', 'WindSpeed3pm']].head(5)

,WindSpeed9am,WindSpeed3pm
0,High,High Average
1,Low,High Average
2,High Average,High
3,Low Average,Low
4,Low,High Average


In [ ]:
df['WindSpeed'] = df.apply(lambda row: determine_category(row, 'WindSpeed9am', 'WindSpeed3pm'), axis=1)

In [ ]:
df[['WindSpeed9am', 'WindSpeed3pm', 'WindSpeed']].head(5)

,WindSpeed9am,WindSpeed3pm,WindSpeed
0,High,High Average,High
1,Low,High Average,Low Average
2,High Average,High,High
3,Low Average,Low,Low
4,Low,High Average,Low Average


##Thêm mới cột Humidity

In [ ]:
quartiles_humidity_9am = calculate_quartiles('Humidity9am', df)
quartiles_humidity_3pm = calculate_quartiles('Humidity3pm', df)

print(f'Giá trị trung bình của cột Humidity9am',quartiles_humidity_9am)
print(f'Giá trị trung bình của cột Humidity3pm',quartiles_humidity_3pm)

df['Humidity9am'] = df.apply(lambda row: temporary_category(row['Humidity9am'], quartiles_humidity_9am), axis=1)
df['Humidity3pm'] = df.apply(lambda row: temporary_category(row['Humidity3pm'], quartiles_humidity_3pm), axis=1)

df[['Humidity9am', 'Humidity3pm']].head(5)

Giá trị trung bình của cột Humidity9am [57. 69. 82.]
Giá trị trung bình của cột Humidity3pm [37.         51.48260609 64.        ]


,Humidity9am,Humidity3pm
0,High Average,Low
1,Low,Low
2,Low,Low
3,Low,Low
4,High Average,Low


In [ ]:
df['Humidity'] = df.apply(lambda row: determine_category(row, 'Humidity9am', 'Humidity3pm'), axis=1)
df[['Humidity9am', 'Humidity3pm', 'Humidity']].head(5)

,Humidity9am,Humidity3pm,Humidity
0,High Average,Low,Low Average
1,Low,Low,Low
2,Low,Low,Low
3,Low,Low,Low
4,High Average,Low,Low Average


##Thêm mới cột Pressure

In [ ]:
quartiles_pressure_9am = calculate_quartiles('Pressure9am', df)
quartiles_pressure_3pm = calculate_quartiles('Pressure3pm', df)

print(f'Giá trị trung bình của cột Pressure9am',quartiles_pressure_9am)
print(f'Giá trị trung bình của cột Pressure3pm',quartiles_pressure_3pm)

df['Pressure9am'] = df.apply(lambda row: temporary_category(row['Pressure9am'], quartiles_pressure_9am), axis=1)
df['Pressure3pm'] = df.apply(lambda row: temporary_category(row['Pressure3pm'], quartiles_pressure_3pm), axis=1)

df[['Pressure9am', 'Pressure3pm']].head(5)

Giá trị trung bình của cột Pressure9am [1014.         1017.65375842 1022.1       ]
Giá trị trung bình của cột Pressure3pm [1011.5        1015.25820354 1019.7       ]


,Pressure9am,Pressure3pm
0,Low,Low
1,Low,Low
2,Low,Low
3,Low Average,Low Average
4,Low,Low


In [ ]:
df['Pressure'] = df.apply(lambda row: determine_category(row, 'Pressure9am', 'Pressure3pm'), axis=1)
df[['Pressure9am', 'Pressure3pm', 'Pressure']].head(5)

,Pressure9am,Pressure3pm,Pressure
0,Low,Low,Low
1,Low,Low,Low
2,Low,Low,Low
3,Low Average,Low Average,Low Average
4,Low,Low,Low


##Thêm mới cột Temp

In [ ]:
quartiles_temp_9am = calculate_quartiles('Temp9am', df)
quartiles_temp_3pm = calculate_quartiles('Temp3pm', df)

print(f'Giá trị trung bình của cột Temp9am',quartiles_temp_9am)
print(f'Giá trị trung bình của cột Temp3pm',quartiles_temp_3pm)

df['Temp9am'] = df.apply(lambda row: temporary_category(row['Temp9am'], quartiles_temp_9am), axis=1)
df['Temp3pm'] = df.apply(lambda row: temporary_category(row['Temp3pm'], quartiles_temp_3pm), axis=1)

df[['Temp9am', 'Temp3pm']].head(5)

Giá trị trung bình của cột Temp9am [12.3 16.7 21.4]
Giá trị trung bình của cột Temp3pm [16.8 21.3 26.4]


,Temp9am,Temp3pm
0,High Average,High Average
1,High Average,High Average
2,High Average,High Average
3,High Average,High
4,High Average,High


In [ ]:
df['Temp'] = df.apply(lambda row: determine_category(row, 'Temp9am', 'Temp3pm'), axis=1)
df[['Temp9am', 'Temp3pm', 'Temp']].head(5)

,Temp9am,Temp3pm,Temp
0,High Average,High Average,High Average
1,High Average,High Average,High Average
2,High Average,High Average,High Average
3,High Average,High,High
4,High Average,High,High


#XÂY DỰNG BÀI TOÁN FP-GROWTH

In [ ]:
# Phân tích tần suất xuất hiện của các biến quan trọng
frequencies = df[["WindSpeed", "Humidity", "Pressure", "Temp", "RainToday", "RainTomorrow"]].apply(pd.Series.value_counts)

# In bảng tần suất xuất hiện của các biến
print("Bảng tần suất xuất hiện của các biến:")
print(frequencies)

Bảng tần suất xuất hiện của các biến:
              WindSpeed  Humidity  Pressure     Temp  RainToday  RainTomorrow
Average         22488.0   20893.0    9840.0  17213.0        NaN           NaN
High            28906.0   35846.0   36250.0  40426.0        NaN           NaN
High Average    18697.0   19957.0   20704.0  16675.0        NaN           NaN
Low             40007.0   37081.0   37681.0  40057.0        NaN           NaN
Low Average     21843.0   18164.0   27466.0  17570.0        NaN           NaN
No                  NaN       NaN       NaN      NaN   105497.0      105150.0
Yes                 NaN       NaN       NaN      NaN    26444.0       26791.0


In [ ]:
# Chuẩn bị dữ liệu cho FP-Growth
transactions = df[["WindSpeed", "Humidity", "Pressure", "Temp",
                   "RainToday", "RainTomorrow"]].apply(lambda x:
                    [f"{col}={val}" for col, val in zip(x.index, x)], axis=1).values.tolist()
transactions[:2]

[['WindSpeed=High',
  'Humidity=Low Average',
  'Pressure=Low',
  'Temp=High Average',
  'RainToday=No',
  'RainTomorrow=No'],
 ['WindSpeed=Low Average',
  'Humidity=Low',
  'Pressure=Low',
  'Temp=High Average',
  'RainToday=No',
  'RainTomorrow=No']]

In [ ]:
# Xây dựng mô hình FP-Growth và tìm các itemsets phổ biến
min_support = int(len(transactions) * 0.12)  # Tính minSup
patterns = pyfpgrowth.find_frequent_patterns(transactions, min_support)

# In các bảng luật kết hợp
for num_items in range(1, len(transactions[0]) + 1):
    itemsets = [itemset for itemset in patterns.keys() if len(itemset) == num_items]
    if not itemsets:
        continue

    # Xây dựng bảng luật kết hợp
    table = PrettyTable()
    table.field_names = ["Itemset", "Support"]
    for itemset in itemsets:
        support = patterns[itemset]
        table.add_row([", ".join(itemset), support])

    # In bảng luật kết hợp
    print(f"\nBảng luật kết hợp ({num_items} phần tử):")
    print(table)


Bảng luật kết hợp (1 phần tử):
+------------------------+---------+
|        Itemset         | Support |
+------------------------+---------+
|   Temp=High Average    |  16675  |
|      Temp=Average      |  17213  |
|    Temp=Low Average    |  17570  |
| WindSpeed=High Average |  18697  |
| Humidity=High Average  |  19957  |
|     RainToday=Yes      |  26444  |
|    RainTomorrow=Yes    |  26791  |
|    RainTomorrow=No     |  105150 |
|      RainToday=No      |  105497 |
+------------------------+---------+

Bảng luật kết hợp (2 phần tử):
+----------------------------------------+---------+
|                Itemset                 | Support |
+----------------------------------------+---------+
|   Humidity=Low Average, RainToday=No   |  16088  |
| Humidity=Low Average, RainTomorrow=No  |  16214  |
|  Pressure=High Average, RainToday=No   |  17293  |
| Pressure=High Average, RainTomorrow=No |  17476  |
|   Humidity=Average, RainTomorrow=No    |  17725  |
|     Humidity=Average, RainTod

In [ ]:
# Xây dựng các quy tắc kết hợp từ các itemsets phổ biến
min_confidence = 0.5  # minConf
rules = pyfpgrowth.generate_association_rules(patterns, min_confidence)

# Lọc các quy tắc có hơn 2 phần tử
filtered_rules = {rule: confidence for rule, confidence in rules.items() if len(rule) >= 3}

# Sắp xếp các quy tắc theo mức độ kết hợp giảm dần
sorted_rules = sorted(filtered_rules.items(), key=lambda x: x[1], reverse=True)

# In ra 50 quy tắc mạnh nhất
print("\nTập hợp các quy tắc mạnh có phần tử trong luật:")
for rule, confidence in sorted_rules[:50]:
    antecedent = rule[:-1]
    consequent = rule[-1]
    print(f"{antecedent} -> {consequent}, Conf = {confidence[1]}")


Tập hợp các quy tắc mạnh có phần tử trong luật:
('Humidity=Low', 'RainToday=No') -> RainTomorrow=No, Conf = 0.5084801531512668
('Humidity=Low', 'RainToday=No') -> Temp=High, Conf = 0.9387045115688332
('Humidity=Low', 'RainTomorrow=No') -> Temp=High, Conf = 0.9776282493673798
('RainToday=No', 'RainTomorrow=No') -> Temp=High, Conf = 0.5477011309082708


Thiếu sự đa dạng của dữ liệu